In [47]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from math import sin, cos, sqrt, atan2, radians, degrees
import copy, glob, os
%matplotlib inline  

In [48]:
trend = []
plunge =[]

os.chdir("/home/blanck/schwarm/relative_relocation_slunga")
for file in glob.glob("*.list"):


    f=open(file,'r')
    data=f.readlines()

    lon_i=[]
    lat_i=[]
    zs=[]
    for x in data:
        lon_i.append(x.split(' ')[2])
        lat_i.append(x.split(' ')[1])
        zs.append(float(x.split()[3]))
    f.close()

    # approxsmate radius of earth in km
    R = 6373.0

    xs=[]
    ys=[]

    lat1 = radians(64.01)
    lon1 = radians(-21.30)

    for i in range(len(zs)):
        #print(float(lat_i[i]))
        lat2 = radians(float(lat_i[i]))
        lon2 = radians(float(lon_i[i]))

        #dx (d_lon; lat is fixed)

        dlon = lon2 - lon1
        dlat = 0

        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))

        dx = R * c

        #dy (d_lat; lon is fixed)

        dlon = 0
        dlat = lat2 - lat1

        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))

        dy = R * c

        #print("Result: dx=", dx, "; dy=", dy)

        xs.append(dx)
        ys.append(dy)

    xs_comp = copy.copy(xs)
    ys_comp = copy.copy(ys)
    zs_comp = copy.copy(zs)

   #print "e_max:"
    # do fit
    e_max = 10
    while e_max > 0.25 and len(xs) > 20:
        tmp_A = []
        tmp_b = []
        for i in range(len(xs)):
            tmp_A.append([xs[i], ys[i], 1])
            tmp_b.append(zs[i])
        b = np.matrix(tmp_b).T
        A = np.matrix(tmp_A)
        fit = (A.T * A).I * A.T * b
        errors = b - A * fit
        residual = np.linalg.norm(errors)
        err = np.asarray(errors)
        e = err.tolist()
        e_max=max(abs(err))
        for i in range(len(e)):
            if np.abs(e[i]) == np.abs(e_max):
                rem = i
        del xs[rem]
        del ys[rem]
        del zs[rem]
        #print(len(xs), len(ys), len(zs))

    #print "e_max:", e_max
    #print "datapoints:", len(xs)

    #print "solution:"
    #print "%f x + %f y + %f = z" % (fit[0], fit[1], fit[2])
    #print "normal vector:"
    #print "(%f, %f, 1)" % (-fit[0], -fit[1])
    
#calculate trend and plunge from normal vector in degrees
    trend.append(degrees(atan2 (fit[0], fit[1])))
    plunge.append(degrees(atan2 (-1, sqrt(fit[0]**2+fit[1]**2))))

print "trend:"
print trend

print "plunge:"
print plunge

trend:
[-158.08099685011965, -162.7160384619961, -162.68106976342082, -124.67144107754392, -158.51948774985394, 164.9328234971961, -162.7160384619961, -140.47040017828198, -131.15077429246523, -168.45826913264915, -159.29709144034814, -163.36350919430572, -163.32205171690134, -59.8467981445624]
plunge:
[-13.144355581036267, -13.208677702281273, -12.278920071641364, -23.58145337530394, -12.994218489184265, -6.662955435158157, -13.208677702281273, -18.89513754443774, -20.328187216749892, -10.97523371783674, -12.72930763131438, -12.230347135026772, -12.281754678376396, -47.92218884028755]


In [20]:
f1 = open("/home/blanck/schwarm/relative_relocation_slunga/trend.txt","w")
for i in range(len(trend)):
    f1.write("1 %s\n" % trend[i] )
f1.close()
        
f2 = open("/home/blanck/schwarm/relative_relocation_slunga/plunge.txt","w")
for i in range(len(plunge)):
    f2.write("1 %s\n" % plunge[i] )
f2.close()

2
